In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 1, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450,
               800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2, 
               1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log. 

In [3]:
def calc_logloss(y, y_pred):
    np_log1 = np.array([(0 if i == 0 else np.log(i)) for i in y_pred])
    np_log2 = np.array([(0 if (1-i) == 0 else np.log(1-i)) for i in y_pred])
    err = - np.mean(y * np_log1 + (1.0 - y) * np_log2)
    return err

In [4]:
y1 = np.array([1, 0])
y_pred1 = np.array([0.9, 0.1])
calc_logloss(y1, y_pred1)

0.10536051565782628

2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
def sigmoid(z):
  res = 1 / (1 + np.exp(-z))
  return res

In [6]:
def calc_std_feat(x):
  res = (x - x.mean()) / x.std()
  return res

In [7]:
X_st = X.copy()
X_st[1, :] = calc_std_feat(X[1, :])
X_st[2, :] = calc_std_feat(X[2, :])
X_st[3, :] = calc_std_feat(X[3, :])
X_st

array([[ 1.        ,  1.        ,  1.        ,  1.        ,  1.        ,
         1.        ,  1.        ,  1.        ,  1.        ,  1.        ],
       [-0.57142857, -0.57142857, -0.21428571, -0.57142857,  0.14285714,
        -0.92857143,  0.85714286,  2.64285714, -0.57142857, -0.21428571],
       [-0.97958969, -0.56713087, -0.46401617, -0.77336028,  0.97958969,
        -0.36090146,  1.08270439,  2.11385144, -1.08270439,  0.05155735],
       [-0.89625816, -0.89625816,  0.38411064, -0.89625816,  0.38411064,
        -0.89625816,  1.66447944,  1.66447944, -0.89625816,  0.38411064]])

In [8]:
def eval_model(X, y, iterations, alpha=1e-4):
  np.random.seed(42)
  W = np.random.randn(X.shape[0])
  n = X.shape[1]
  for i in range(1, iterations+1):
    z = np.dot(W, X)
    y_pred = sigmoid(z)
    err = calc_logloss(y, y_pred)
    W -= alpha * (1/n * np.dot((y_pred - y), X.T))
    if i % (iterations / 10) == 0:
      print(i, W, err)
  return W

In [9]:
W = eval_model(X_st, y, iterations=600000, alpha=1e-2)

60000 [ 1.7544713  -4.92444073 -3.93698659 10.30087205] 0.21122487031163897
120000 [ 2.78412922 -6.55109633 -5.9329403  14.5456458 ] 0.1679171096246014
180000 [ 3.51769676 -7.76281218 -7.38541676 17.67889827] 0.14456372021645544
240000 [ 4.08735815 -8.72719965 -8.53723633 20.17462008] 0.12983117497450516
300000 [ 4.55457429 -9.52926692 -9.50080175 22.26190276] 0.1195644022937121
360000 [  4.95235878 -10.21733509 -10.33687602  24.06767795] 0.11190072368205042
420000 [  5.30027238 -10.82118754 -11.08162531  25.66882453] 0.10588788057213452
480000 [  5.61077777 -11.36037419 -11.75814014  27.11511059] 0.10098996551793918
540000 [  5.89226934 -11.84838474 -12.38196815  28.44045241] 0.09688252647464239
600000 [  6.15065392 -12.29491105 -12.96401973  29.66892072] 0.093357571841202


3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [10]:
def calc_pred_proba(W, X):
    result = sigmoid(W.dot(X))
    return result

In [11]:
y_pred_proba = calc_pred_proba(W, X_st)
y_pred_proba

array([0.32843434, 0.0023234 , 1.        , 0.03264616, 0.95651856,
       0.01280639, 1.        , 0.01257062, 0.65056104, 1.        ])

4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [12]:
def calc_pred(W, X, prediction_lvl=0.5):
    y_pred_proba = calc_pred_proba(W, X)
    return np.array([1 if i>prediction_lvl else 0 for i in y_pred_proba])

In [13]:
y_pred = calc_pred(W, X_st)
y_pred

array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1])

In [14]:
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [15]:
train_accuracy = 100.0 - np.mean(np.abs(y_pred - y)*100.0)
train_accuracy

100.0

In [16]:
a = zip(y, y_pred)
y

array([0., 0., 1., 0., 1., 0., 1., 0., 1., 1.])

In [17]:
true_positive = sum([1 if y[i] == y_pred[i] == 1 else 0 for i in range(len(y))])
false_positive = sum([1 if y[i] == 1 and y_pred[i] == 0 else 0 for i in range(len(y))])
true_negative = sum([1 if y[i] == y_pred[i] == 0 else 0 for i in range(len(y))])
false_negative = sum([1 if y[i] == 0 and y_pred[i] == 1 else 0 for i in range(len(y))])
np.array([[true_positive, false_positive], [false_negative, true_negative]])

array([[5, 0],
       [0, 5]])

In [18]:
precision = true_positive / (true_positive + false_positive)
precision

1.0

In [19]:
recall = true_positive / (true_positive + false_negative)
recall

1.0

In [20]:
f1 = 2 * precision * recall / (precision + recall)
f1

1.0

6. Могла ли модель переобучиться? Почему?

Могла. Мало наблюдений.

7*. Создайте функции eval_model_l1 и eval_model_l2 с применением L1 и L2 регуляризаций соответственно.

In [2]:
def eval_model_l1(X, y, iterations, alpha=1e-4, lambda_=1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + (lambda_ * np.sign(W)))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [22]:
def eval_model_l2(X, y, iterations, alpha=1e-4, lambda_=1e-6):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations+1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1/n * np.dot((y_pred - y), X.T) + (lambda_/2) * np.sum(W**2))
        if i % (iterations / 10) == 0:
            print(i, W, err)
    return W

In [23]:
W = eval_model_l1(X_st, y, iterations=600000, alpha=1e-2)
W

60000 [ 1.75410796 -4.92423206 -3.93660601 10.29997208] 0.21123550179295109
120000 [ 2.78326409 -6.55042972 -5.93187558 14.54317684] 0.16793754220183124
180000 [ 3.51624783 -7.76160329 -7.38353214 17.67454017] 0.14459151101660433
240000 [ 4.08525783 -8.72539515 -8.53441323 20.16811472] 0.12986508029194924
300000 [ 4.5517636  -9.52682863 -9.49692623 22.25302034] 0.11960379274829944
360000 [  4.94878543 -10.21423475 -10.33183625  24.05620705] 0.11194527182794396
420000 [  5.29588959 -10.81740376 -11.07531073  25.65456711] 0.10593742078612367
480000 [  5.60554341 -11.35589033 -11.75044123  27.09787894] 0.1010444259783216
540000 [  5.88614523 -11.8431872  -12.37277655  28.42006725] 0.09694189284187724
600000 [  6.14360532 -12.28898786 -12.95322857  29.64520989] 0.09342186609897418


array([  6.14360532, -12.28898786, -12.95322857,  29.64520989])

In [24]:
W = eval_model_l2(X_st, y, iterations=600000, alpha=1e-2)
W

60000 [ 1.7521764  -4.92336451 -3.93482996 10.295602  ] 0.21128680291141463
120000 [ 2.77664012 -6.54622316 -5.92453046 14.52572891] 0.1680815629585433
180000 [ 3.50250887 -7.75199464 -7.36723094 17.63614562] 0.14483660596335884
240000 [ 4.0622084  -8.70868526 -8.5059244  20.10149928] 0.13021408033853474
300000 [ 4.51738314 -9.50163969 -9.45307839 22.15135603] 0.12005905245648944
360000 [  4.90119504 -10.17947254 -10.26946816  23.91300921] 0.11250958698188607
420000 [  5.23332842 -10.77222494 -10.9912432   25.46362095] 0.10661414605395851
480000 [  5.52635062 -11.29967436 -11.64146279  26.85318753] 0.1018374188047306
540000 [  5.78874538 -11.77551176 -12.23563852  28.11581494] 0.09785542550512859
600000 [  6.02649591 -12.20960735 -12.78464658  29.27573697] 0.09446054028138044


array([  6.02649591, -12.20960735, -12.78464658,  29.27573697])